# Reference design
**Authors** : S. Gaydier, I. Zehavi, T. Cherrière, P. Gangl

In [15]:
from shapeOptInductor import gen_mesh9
import ngsolve as ngs
from ngsolve.webgui import Draw
import numpy as np


In [16]:
from netgen.geom2d import SplineGeometry
from debug import *
def gen_mesh(air_gap=4.11e-3, maxh=2e-3, debug=False):
    """Gives a triangular mesh"""
    # Global domain
    r = 0.04

    # Geometry definition
    a = 1e-2
    h = 25e-3
    ba = 1e-2
    print(h / 2 - a / 2)
    print(h / 2)
    geo = SplineGeometry()
    pnts = [
        (0, 0),  # p0
        (a / 2, 0),  # p1
        (a / 2 + ba, 0),  # p2
        (r, 0),  # p3
        (r, r),  # p4
        (0, r),  # p5
        (0, h/2),  # p6
        (0, air_gap / 2),  # p7
        (a / 2, air_gap / 2),  # p8
        (a / 2, h / 2 - a / 2),  # p9
        (a / 2 + ba, h / 2 - a / 2),  # p10
        (a / 2 + ba, air_gap / 2),  # p11
        (a + ba, air_gap / 2),  # p12
        (a + ba, h / 2),  # p13
        ]

    (
        p0,
        p1,
        p2,
        p3,
        p4,
        p5,
        p6,
        p7,
        p8,
        p9,
        p10,
        p11,
        p12,
        p13,
    ) = [geo.AppendPoint(*pnt) for pnt in pnts]

    # List of lines with boundary conditions and domains
    lines = [
        [["line", p0, p1], {"bc": "a", "leftdomain": 3, "rightdomain": 0}],
        [["line", p1, p2], {"bc": "b", "leftdomain": 2, "rightdomain": 0}],
        [["line", p2, p3], {"bc": "c", "leftdomain": 4, "rightdomain": 0}],
        # [["line", p3, p4], {"bc": "d", "leftdomain": 4, "rightdomain": 0}],
        # [["line", p4, p5], {"bc": "d", "leftdomain": 4, "rightdomain": 0}],
        [["spline3", p3, p4, p5], {"bc": "d", "leftdomain": 4, "rightdomain": 0}],
        [["line", p5, p6], {"bc": "e", "leftdomain": 4, "rightdomain": 0}],
        [["line", p6, p7], {"bc": "f", "leftdomain": 1, "rightdomain": 0}],
        [["line", p7, p0], {"bc": "g", "leftdomain": 3, "rightdomain": 0}],
        [["line", p7, p8], {"bc": "h", "leftdomain": 1, "rightdomain":3 }],
        [["line", p8, p9], {"bc": "i", "leftdomain": 1, "rightdomain":2 }],
        [["line", p9, p10], {"bc": "j", "leftdomain": 1, "rightdomain":2 }],
        [["line", p10, p11], {"bc": "k", "leftdomain": 1, "rightdomain":2 }],
        [["line", p11, p12], {"bc": "l", "leftdomain": 1, "rightdomain": 4}],
        [["line", p12, p13], {"bc": "m", "leftdomain": 1, "rightdomain": 4}],
        [["line", p13, p6], {"bc": "n", "leftdomain": 1, "rightdomain": 4}],
        [["line", p1, p8], {"bc": "o", "leftdomain": 3, "rightdomain": 2}],
        [["line", p2, p11], {"bc": "p", "leftdomain": 2, "rightdomain": 4}],
        
    ]

    # Append all lines to the geometry
    for line, props in lines:
        geo.Append(line, **props)

    # Optional matpotlib debug render
    if debug : 
        fig, axes = plt.subplots(2, 2, figsize=(15, 15))
        debug_geo(geo, axes[0, 0])
        debug_points(geo, 
                     # point_names is optional
                     point_names="p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13".split(","),
                     ax=axes[1, 0]
                     )
        debug_bc_names(geo, ax=axes[0, 1])
        debug_region_labels(geo, ax=axes[1, 1])
        plt.show()

    # Set materials and meshing parameters
    geo.SetMaterial(1, "core")
    geo.SetMaterial(2, "coil")
    geo.SetMaterial(3, "air")
    geo.SetMaterial(4, "air")
    ngmesh = geo.GenerateMesh(maxh=maxh)
    return ngs.Mesh(ngmesh)


## 1 - Geometry and meshing

In [17]:
airgap = 4e-3  # size of the airgap
lz = 1e-2  # thickness in the z-axis
maxh = 2e-3  # maximum element size
s = 4  # symmetry factor (we simulate only one quarter of the inductor)

# generate the geometry (cf shapOpt.py for details)
mesh = gen_mesh9(airgap, maxh)

# define the characteristic functions of all regions
XiAir = mesh.MaterialCF({"air": 1})
XiCore = mesh.MaterialCF({"core": 1})
XiCoil = mesh.MaterialCF({"coil": 1})

# draw the meshed geometry
Draw(1 * XiAir + 2 * XiCoil + 3 * XiCore, mesh, radius=0.02)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## 2 - Computation of (complex) magnetic state

### a) Parameters definition

In [18]:
f = 5e4  # working frequency (Hz)
omega = 2 * np.pi * f  # rad/s
mu0 = 4e-7 * np.pi  # void permeability (H/m)
mur = 1000  # relative permeability of iron (no unit)
mu_iron = mur * mu0  # permeability of iron (H/m)
delta = 0.1  # loss angle associated with the coil (rad)
mu_coil = np.exp(-1j * delta) * mu0  # complex permeability
nb_turn = 200  # number of turn in the coil (no unit)
Is = 2  # source current intensity (A)
js = nb_turn / 2 * Is / (ngs.Integrate(XiCoil, mesh)) * XiCoil  # source current density (A/m²)


### b) Processing

We want to find $\underline{a}\in H^1(\Omega, \mathbb C)$ solving the following equality for any $a^* \in H^1(\Omega, \mathbb R)$, such that $\underline{a} = 0$ on the outer air box boundary and vertical symmetry axis.

$$ \int_\Omega \nabla a^* \cdot \left( \frac{\chi_c}{\underline{\mu_f}} + \frac{\chi_f}{\underline{\mu_f}} +\frac{\chi_a}{\underline{\mu_0}}\right) \nabla \underline{a} = \int_\Omega \chi_c j$$

In [21]:
def magWeakFormComplex(a, a_):
    """Return the complex weak form of the magnetic problem, i.e, bilinear
    and linear forms (matrix and right-hand side after discretization, respectively)"""

    # bilinear form (transmission)
    bf = ngs.grad(a_) * 1 / mu_iron * ngs.grad(a) * ngs.dx("core")
    bf += ngs.grad(a_) * 1 / mu_coil * ngs.grad(a) * ngs.dx("coil")
    bf += ngs.grad(a_) * 1 / mu0 * ngs.grad(a) * ngs.dx("air")

    # linear form (source)
    lf = a_ * js * ngs.dx("coil")

    return bf, lf


def solveStateComplex(mesh):
    """Solve the complex magnetic state"""
    # definition of function space
    fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert", complex=True)
    # fes = ngs.H1(mesh, order=1, dirichlet="d|e|f|g", complex=True)
    a, a_ = fes.TnT()

    # definition of weak form
    bf, f = magWeakFormComplex(a, a_)
    K, F = ngs.BilinearForm(fes), ngs.LinearForm(fes)
    K += bf
    F += f

    # assembly
    K.Assemble()
    F.Assemble()

    # solving
    gf = ngs.GridFunction(fes)
    Kinv = K.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")
    gf.vec.data = Kinv * F.vec

    return gf


# we solve and plot for the given geometry, and plot the vector potential (equipotential are flux-lines).
a = solveStateComplex(mesh)
Draw(ngs.Norm(a), mesh, animate_complex=True)
Draw(a, mesh, animate_complex=True)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 's…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 's…

BaseWebGuiScene

### c) Post-processing

The AC losses in conductors are computed from the imaginary part of their permeability (or in this case, reluctivity).
The current is assumed to be sinus. In reality, it would rather be triangular, which adds harmonics that are neglected here. Since we assume the linearity of the problem linear, one needs to solve the system for each frequency with the corresponding current harmonics, and sum the losses (which is not done here for simplicity and speed).
We compute the losses from the imaginary part of the reluctivity (in the coil)

$$ P(\underline{a}) = s l_z \pi f \int_\Omega \Im \left( \frac{\chi_c}{\underline{\mu_c}} \right) |\nabla \underline{a}|^2 \; \mathrm{d} x, $$

and the inductance from the real part of the reluctivity (everywhere)

$$ L(\underline{a}) = \frac{s l_z}{I^2} \int_\Omega \Re \left( \frac{\chi_c}{\underline{\mu_f}} + \frac{\chi_f}{\underline{\mu_f}} +\frac{\chi_a}{\underline{\mu_0}}\right) |\nabla \underline{a}|^2 \; \mathrm{d} x. $$

In [20]:
def Losses(a, mesh):
    rel = XiCoil / mu_coil
    return s * np.pi * f * lz * ngs.Integrate(rel.imag * ngs.Norm(ngs.grad(a)) ** 2, mesh)


def Inductance(a, mesh):
    rel = XiAir / mu0 + XiCoil / mu_coil + XiCore / mu_iron
    return s * lz / (Is**2) * ngs.Integrate(rel.real * ngs.Norm(ngs.grad(a)) ** 2, mesh)


print(f" P_AC = {Losses(a,mesh) :.2f} W")
print(f" L = {Inductance(a,mesh) * 1e3 :.2f} mH")


 P_AC = 13.39 W
 L = 1.02 mH
